In [38]:
import torch
from torch import tensor
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from ast import literal_eval
from py_experimenter.experimenter import PyExperimenter
import seaborn as sns
import torch
import mysql.connector
import openml


from py_experimenter.database_connector_mysql import DatabaseConnectorMYSQL

def connect(self):

    db = mysql.connector.connect(
        host="xxx",
        user="xxx",
        password="xxx",
        database="xxx",
        ssl_disabled=False,
    )
    return db


def _start_transaction(self, connection, readonly=False):
    if not readonly:
        connection.start_transaction()


DatabaseConnectorMYSQL.connect = connect
DatabaseConnectorMYSQL._start_transaction = _start_transaction

%matplotlib inline

In [39]:
experimenter = PyExperimenter(
    experiment_configuration_file_path="./experiments/config/config.yml",
)
exp_frame = experimenter.get_table()


2024-08-19 16:56:50,196  | py-experimenter - INFO     | Found 5 keyfields
2024-08-19 16:56:50,197  | py-experimenter - INFO     | Found 9 resultfields
2024-08-19 16:56:50,198  | py-experimenter - WARNING  | No logtables given
2024-08-19 16:56:50,198  | py-experimenter - WARNING  | No custom section defined in config
2024-08-19 16:56:50,198  | py-experimenter - WARNING  | No codecarbon section defined in config
2024-08-19 16:56:50,208  | py-experimenter - INFO     | Initialized and connected to database


In [40]:
exp_frame

,ID,model,openml_id,master_seed,alpha,fraction_cal_samples,creation_date,status,start_date,name,...,score_bacc,coverage_mean,coverage_std,efficiency_mean,efficiency_std,clf_seed,mccv_seed,end_date,error,rn
0,62,classifier_nn,15,1,0.05,0.1,2024-08-14 16:50:38,done,2024-08-14 16:50:39,PyExperimenter,...,0.973118,0.957143,0.202535,0.992857,0.084213,1095513148,3280387012,2024-08-14 16:51:17,None,1
1,569,classifier_nn,15,1,0.05,0.2,2024-08-14 16:50:38,done,2024-08-14 17:16:53,PyExperimenter,...,0.973118,0.964286,0.185577,1.000000,0.000000,1095513148,3280387012,2024-08-14 17:17:19,None,1
2,225,classifier_nn,15,1,0.10,0.1,2024-08-14 16:50:38,done,2024-08-14 16:58:28,PyExperimenter,...,0.973118,0.921429,0.269069,0.950000,0.217945,1095513148,3280387012,2024-08-14 16:58:55,None,1
3,741,classifier_nn,15,1,0.10,0.2,2024-08-14 16:50:38,done,2024-08-14 17:41:12,PyExperimenter,...,0.973118,0.928571,0.257539,0.950000,0.217945,1095513148,3280387012,2024-08-14 17:41:46,None,1
4,401,classifier_nn,15,1,0.20,0.1,2024-08-14 16:50:38,done,2024-08-14 17:07:43,PyExperimenter,...,0.973118,0.821429,0.382993,0.835714,0.370535,1095513148,3280387012,2024-08-14 17:08:10,None,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1795,731,random_forest,4534,10,0.05,0.2,2024-08-14 16:50:38,done,2024-08-14 17:41:11,PyExperimenter,...,0.967596,0.951606,0.214598,0.971054,0.167655,1842064464,2454155475,2024-08-14 17:41:26,None,1
1796,395,random_forest,4534,10,0.10,0.1,2024-08-14 16:50:38,done,2024-08-14 17:07:19,PyExperimenter,...,0.967915,0.869742,0.336587,0.876526,0.328980,1842064464,2454155475,2024-08-14 17:07:32,None,1
1797,911,random_forest,4534,10,0.10,0.2,2024-08-14 16:50:38,done,2024-08-14 17:42:00,PyExperimenter,...,0.967596,0.893713,0.308204,0.902759,0.296286,1842064464,2454155475,2024-08-14 17:42:13,None,1
1798,563,random_forest,4534,10,0.20,0.1,2024-08-14 16:50:38,done,2024-08-14 17:16:30,PyExperimenter,...,0.967915,0.803256,0.397537,0.807327,0.394398,1842064464,2454155475,2024-08-14 17:16:44,None,1


In [41]:
import numpy as np
datasets = {}
num_classes = {}
dataset_renamer = {}
for id in exp_frame.openml_id.unique():
    dataset = openml.datasets.get_dataset(id.item())
    target_attribute = dataset.default_target_attribute
    X, y, _, _ = dataset.get_data(target=target_attribute)
    datasets[id] = dataset
    dataset_renamer[id] = dataset.name
    num_classes[id] = len(np.unique(y))

/home/jonas/Documents/Research/dyad_ranking/torch_plnet/.venv/lib/python3.11/site-packages/openml/utils.py:461: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  return func(*args, **kwargs)
/home/jonas/Documents/Research/dyad_ranking/torch_plnet/.venv/lib/python3.11/site-packages/openml/utils.py:461: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  return func(*args, 

In [87]:
df = exp_frame

exp_frame["dataset"] =  exp_frame["openml_id"].replace(dataset_renamer)
exp_frame["num_classes"] = exp_frame["openml_id"].replace(num_classes)

exp_frame["dataset"] = exp_frame["dataset"] + " (" + exp_frame["num_classes"].astype(str) + ")" 

group_cols = ['dataset', 'alpha', 'fraction_cal_samples', 'model']
metrics_max = ['score_bacc', 'score_acc', 'coverage_mean']
metrics_min = ['efficiency_mean']
metrics = metrics_max + metrics_min

grouped_df = df.groupby(group_cols).agg({metric: 'mean' for metric in metrics}).reset_index()
agg_dict = {metric: 'max' for metric in metrics_max}
agg_dict.update({metric: 'min' for metric in metrics_min})
best_values = grouped_df.groupby(['dataset', 'alpha', 'fraction_cal_samples']).agg(agg_dict).reset_index()

In [88]:
def generate_latex_table_with_lines(df, group_cols, value_cols, best_values):
    """Generate a LaTeX table with multirow entries, bold the best values, and add lines between consecutive multirows."""
    
    def format_value(value, group, metric, max=True):
        """Format the value, making the highest value bold."""
        max_value = best_values.loc[
            (best_values['dataset'] == group['dataset']) & 
            (best_values['alpha'] == group['alpha']) & 
            (best_values['fraction_cal_samples'] == group['fraction_cal_samples']), metric].values[0]
        if value == max_value:
            return f"\\textbf{{{value:.4f}}}"
        else:
            return f"{value:.4f}"
    
    def recursive_build(df, group_cols):
        # if len(group_cols) == 1:
            # Base case: only one group left, print it directly
            latex_str = ""
            prev_row = None
            for row_id, row in df.iterrows():
                latex_str_tmp = ""
                col_id_list = []
                for col_id, col in enumerate(group_cols):
                    if prev_row is None or row[col] != prev_row[col]:
                        col_id_list.append(col_id)
                        filter_cols = group_cols[:col_id+1]
                        indices = []
                        values = []
                        for fcol in filter_cols:
                            indices.append(fcol)
                            values.append(row[fcol])
                        criteria = dict(zip(indices, values))
                        # Filter the DataFrame using query
                        mask = pd.Series([True] * len(df))

                        # Iterate over criteria to apply conditions
                        for key, value in criteria.items():
                            mask = mask & (df[key] == value)

                        # Filter the DataFrame
                        filtered_df = df[mask]

                        length = len(filtered_df)


                        latex_str_tmp += f" \\multirow{{{length}}}{{*}}{{{row[col]}}} & "
                        if col_id == len(group_cols) - 1:
                            latex_str_tmp += " & ".join([format_value(row[col], row, col) for col in value_cols]) + " \\\\\n"

                        # if prev_row is not None and col_id < len(group_cols) - 1:
                        # latex_str_tmp += " \\\\ \cline{%d-%d} \\\\" % (col_id + 1, len(group_cols) + len(value_cols))

                    else:
                        latex_str_tmp += " & "
                        if col_id == len(group_cols) - 1:
                            latex_str_tmp += " & ".join([format_value(row[col], row, col) for col in value_cols]) + " \\\\\n"
                if row_id > 0 and group_cols[min(col_id_list)] != "model":
                    latex_str += "\\cline{%d-%d}" % (min(col_id_list)+1, len(group_cols) + len(value_cols))
                latex_str += latex_str_tmp
                indices = []
                values = []
                prev_row = row
            return latex_str
        
    # Start recursive building
    latex_body = recursive_build(df, group_cols)

    # Complete LaTeX table
    num_columns = len(group_cols) + len(value_cols)
    col_format = 'l' * len(group_cols) + 'r' * len(value_cols)
    latex_table = f"""
\\begin{{tabular}}{{{col_format}}}
\\toprule
{' & '.join(group_cols)} & {' & '.join(value_cols)} \\\\
\\midrule
{latex_body}
\\bottomrule
\\end{{tabular}}
"""
    return latex_table

# Generate the LaTeX table with bold formatting and lines
latex_table = generate_latex_table_with_lines(grouped_df, group_cols, metrics, best_values)

# Print or save the LaTeX table
print(latex_table.replace("_", "\_"))



\begin{tabular}{llllrrrr}
\toprule
dataset & alpha & fraction\_cal\_samples & model & score\_bacc & score\_acc & coverage\_mean & efficiency\_mean \\
\midrule
 \multirow{18}{*}{PhishingWebsites (2)} &  \multirow{6}{*}{0.05} &  \multirow{3}{*}{0.1} &  \multirow{1}{*}{classifier\_nn} & 0.9624 & 0.9630 & \textbf{0.9557} & 0.9867 \\
 &  &  &  \multirow{1}{*}{plnet} & 0.9294 & 0.9328 & 0.9510 & 1.3393 \\
 &  &  &  \multirow{1}{*}{random\_forest} & \textbf{0.9715} & \textbf{0.9724} & 0.9533 & \textbf{0.9681} \\
\cline{3-8} &  &  \multirow{3}{*}{0.2} &  \multirow{1}{*}{classifier\_nn} & 0.9600 & 0.9607 & 0.9536 & 0.9883 \\
 &  &  &  \multirow{1}{*}{plnet} & 0.9602 & 0.9607 & 0.9503 & 1.2176 \\
 &  &  &  \multirow{1}{*}{random\_forest} & \textbf{0.9701} & \textbf{0.9709} & \textbf{0.9552} & \textbf{0.9734} \\
\cline{2-8} &  \multirow{6}{*}{0.1} &  \multirow{3}{*}{0.1} &  \multirow{1}{*}{classifier\_nn} & 0.9624 & 0.9630 & 0.9044 & 0.9141 \\
 &  &  &  \multirow{1}{*}{plnet} & 0.9294 & 0.9328 &

In [89]:
grouped_df

,dataset,alpha,fraction_cal_samples,model,score_bacc,score_acc,coverage_mean,efficiency_mean
0,PhishingWebsites (2),0.05,0.1,classifier_nn,0.962440,0.963048,0.955721,0.986657
1,PhishingWebsites (2),0.05,0.1,plnet,0.929424,0.932836,0.950972,1.339303
2,PhishingWebsites (2),0.05,0.1,random_forest,0.971482,0.972365,0.953279,0.968069
3,PhishingWebsites (2),0.05,0.2,classifier_nn,0.959992,0.960742,0.953596,0.988331
4,PhishingWebsites (2),0.05,0.2,plnet,0.960163,0.960651,0.950339,1.217638
...,...,...,...,...,...,...,...,...
175,wine (3),0.20,0.1,plnet,0.989167,0.986111,0.927778,1.430555
176,wine (3),0.20,0.1,random_forest,0.982083,0.977778,0.825000,0.827778
177,wine (3),0.20,0.2,classifier_nn,0.979583,0.977778,0.780556,0.780556
178,wine (3),0.20,0.2,plnet,0.981816,0.977778,0.933333,1.566665
